In [1]:
using Pkg
Pkg.activate("../")

using Plots

using LinearAlgebra
using SparseArrays
using BenchmarkTools

using Revise
using MarkovModels

  Activating environment at `~/Repositories/MarkovModels.jl/Project.toml`
┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1317
┌ Info: Precompiling MarkovModels [c2ae6250-d0a1-11ea-0991-234599ce5244]
└ @ Base loading.jl:1317


In [3]:
S = 1000 # Number of states
N = 100 # Number of frames
SF = LogSemifield{Float64}  # Semi-Field for inference
lhs = rand(S, N) # simulated likelihoods

# Construct a HMM with S states and a left-to-right topology.
function make_hmm(S)
    pinit = zeros(SF, S)
    pinit[1] = one(SF)
    pfinal = zeros(SF, S)
    pfinal[3] = one(SF)
    prob = ones(SF, S) .* SF(log(1/2))
    A = diagm(0 => prob, 1 => prob[1:end-1])
    (π=pinit, ω=pfinal, A=A)
end

# Construct a HMM with S states and a left-to-right topology.
# Use a sparse matrix for the transition.
function make_hmm_sparse(S)
    pinit = spzeros(SF, S)
    pinit[1] = one(SF)
    pfinal = spzeros(SF, S)
    pfinal[3] = one(SF)
    prob = ones(SF, S) .* SF(log(1/2))
    A = spdiagm(0 => prob, 1 => prob[1:end-1])
    dists = MarkovModels.calculate_distances(pfinal, A)
    (π=pinit, ω=pfinal, A=A, dists=dists)
end

hmm = make_hmm(S)
sphmm = make_hmm_sparse(S);

In [27]:
test_lhs = rand(3, 10)
hmm = make_hmm(3)
sphmm = make_hmm_sparse(3);
display(αβrecursion(hmm.π, hmm.ω, hmm.A, convert(Matrix{SF}, test_lhs)))
display(αβrecursion(sphmm.π, sphmm.ω, sphmm.A, convert(Matrix{SF}, test_lhs), 
                    pruning = SF(Inf), distances = sphmm.dists))

(SemifieldAlgebra.Semifield{Float64, LogExpFunctions.logaddexp, +, -, -Inf, 0}[0.0 -0.051498712563327054 … -Inf -Inf; -Inf -2.9918373242276095 … -1.6496239631702703 -Inf; -Inf -Inf … -0.21334439530263882 0.0], 4.505753122328786)

(
  0.0  -0.0514987  -0.166238  -0.380165  …   -4.20085        ⋅     ⋅  
  ⋅      -2.99184   -1.89634   -1.18414     -0.816043   -1.64962   ⋅  
  ⋅           ⋅     -5.79659   -4.58115     -0.610942  -0.213344   0.0, 4.505753122328786)

In [23]:
hmm = make_hmm(S)
sphmm = make_hmm_sparse(S);

In [6]:
@benchmark αβrecursion(hmm.π, hmm.ω, hmm.A, convert(Matrix{SF}, lhs))

BenchmarkTools.Trial: 
  memory estimate:  6.16 MiB
  allocs estimate:  220
  --------------
  minimum time:     12.346 s (0.00% GC)
  median time:      12.346 s (0.00% GC)
  mean time:        12.346 s (0.00% GC)
  maximum time:     12.346 s (0.00% GC)
  --------------
  samples:          1
  evals/sample:     1

In [25]:
@benchmark αβrecursion(sphmm.π, sphmm.ω, sphmm.A, convert(Matrix{SF}, lhs), 
                       pruning = SF(10), distances = sphmm.dists)

BenchmarkTools.Trial: 
  memory estimate:  23.46 MiB
  allocs estimate:  649509
  --------------
  minimum time:     78.383 ms (0.00% GC)
  median time:      120.188 ms (3.22% GC)
  mean time:        112.430 ms (2.28% GC)
  maximum time:     145.593 ms (3.82% GC)
  --------------
  samples:          45
  evals/sample:     1

In [26]:
@benchmark αβrecursion(sphmm.π, sphmm.ω, sphmm.A, convert(Matrix{SF}, lhs), 
                       pruning = SF(Inf), distances = sphmm.dists)

BenchmarkTools.Trial: 
  memory estimate:  25.40 MiB
  allocs estimate:  650336
  --------------
  minimum time:     174.870 ms (0.00% GC)
  median time:      213.918 ms (2.29% GC)
  mean time:        210.286 ms (1.81% GC)
  maximum time:     225.599 ms (2.20% GC)
  --------------
  samples:          24
  evals/sample:     1

In [152]:
αβrecursion(sphmm.π, sphmm.ω, sphmm.A, convert(Matrix{SF}, lhs), 
            pruning = SF(1), distances = sphmm.dists)

1000×1000 SparseMatrixCSC{SemifieldAlgebra.Semifield{Float64, LogExpFunctions.logaddexp, +, -, -Inf, 0}, Int64} with 2997 stored entries:
⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀

In [131]:
@benchmark αβrecursion(sphmm.π, sphmm.ω, sphmm.A, convert(Matrix{SF}, lhs), 
                       pruning = SF(100), distances = sphmm.dists)

BenchmarkTools.Trial: 
  memory estimate:  1.88 GiB
  allocs estimate:  6568963
  --------------
  minimum time:     1.992 s (5.06% GC)
  median time:      2.007 s (5.24% GC)
  mean time:        2.017 s (5.40% GC)
  maximum time:     2.051 s (5.88% GC)
  --------------
  samples:          3
  evals/sample:     1

In [132]:
@benchmark αβrecursion(sphmm.π, sphmm.ω, sphmm.A, convert(Matrix{SF}, lhs), 
                       pruning = SF(1000), distances = sphmm.dists)

BenchmarkTools.Trial: 
  memory estimate:  2.82 GiB
  allocs estimate:  7064008
  --------------
  minimum time:     3.287 s (4.34% GC)
  median time:      3.537 s (4.47% GC)
  mean time:        3.537 s (4.47% GC)
  maximum time:     3.787 s (4.59% GC)
  --------------
  samples:          2
  evals/sample:     1

In [181]:
lnαβ = αβrecursion(cfsm, llh, pruning = ThresholdPruning(50))
γ = resps(fsm, lnαβ)

p = plot()
plot!(p, γ[1], label = "p(z = 1|X)")
plot!(p, γ[2], label = "p(z = 2|X)")
plot!(p, γ[3], label = "p(z = 3|X)")

LoadError: TypeError: in keyword argument pruning, expected Number, got a value of type ThresholdPruning

In [150]:
using SparseArrays

L = eltype(cfsm.A)
L_llh = convert(Matrix{L}, llh[cfsm.pdfidx_map, :])

LoadError: type CompiledFSM has no field pdfidx_map

In [51]:
S = 3
spzeros(S,S)


2×2 SparseMatrixCSC{MarkovModels.Semifield{Float64,StatsFuns.logaddexp,+,-,-Inf,0.0},Int64} with 2 stored entries:
  [1, 1]  =  -Inf
  [2, 2]  =  -Inf

In [28]:
α = sparse(zeros(L, MarkovModels.nstates(cfsm), size(L_llh, 2)))
Aᵀ = transpose(cfsm.A)
α[:, 1] = L_llh[:, 1] .* cfsm.π
for n in 2:N
    α[:, n] = (Aᵀ * α[:, n-1]) .* L_llh[:, n]
end
α

3×5 SparseMatrixCSC{MarkovModels.Semifield{Float64,StatsFuns.logaddexp,+,-,-Inf,0.0},Int64} with 12 stored entries:
  [2, 1]  =  1.99648
  [2, 2]  =  0.273776
  [3, 2]  =  0.102643
  [1, 3]  =  -0.834499
  [2, 3]  =  -0.663367
  [3, 3]  =  2.32829
  [1, 4]  =  0.267885
  [2, 4]  =  -2.76521
  [3, 4]  =  0.478586
  [1, 5]  =  -0.356424
  [2, 5]  =  -4.19355
  [3, 5]  =  0.386524

```python
    def _baum_welch_backward(self, llhs):
        log_trans_mat = self.trans_log_probs
        log_betas = torch.zeros_like(llhs) - float('inf')
        log_betas[-1] = self.final_log_probs
        for i in reversed(range(llhs.shape[0]-1)):
            log_betas[i] = torch.logsumexp(log_trans_mat + llhs[i+1] + \
                           log_betas[i+1], dim=1).view(-1)
        return log_betas
```

In [24]:
β = sparse(zeros(L, MarkovModels.nstates(cfsm), size(L_llh, 2)))
A = cfsm.A
β[:, end] = L_llh[:, 1] .* cfsm.ω
for n in N-1:-1:1
    β[:, n] = (A * β[:, n+1] .* L_llh[:, n])
end
β

3×5 Array{MarkovModels.Semifield{Float64,StatsFuns.logaddexp,+,-,-Inf,0.0},2}:
 1.84497  -0.458353  0.264346  0.201489  1.30333
  5.6821    2.77162    0.4675      -Inf     -Inf
 1.25362    2.59154   3.44464  0.404642     -Inf

In [27]:
γ = α .* β ./ sum(γ, dims=1)
#exp.(Matrix{Float64}(γ))

3×5 Array{MarkovModels.Semifield{Float64,StatsFuns.logaddexp,+,-,-Inf,0.0},2}:
 -Inf       -Inf     -6.34738  -0.921333   0.0
  0.0   -0.53288      -5.9731       -Inf  -Inf
 -Inf  -0.884095  -0.00430692  -0.507478  -Inf

In [26]:
using SparseArrays

In [35]:
state_set = Set( s for s in filter(isemitting, collect(states(fsm))))
S = length(state_set)

A = sparse(zeros(LogSemiring{Float64}, S, S))

for s in state_set
    for (ns, weightpath) in MarkovModels.nextemittingstates(s)
        A[s.pdfindex, ns.pdfindex] = weightpath
    end
end

A 
Aᵀ = transpose(A)

3×3 Transpose{MarkovModels.Semiring{Float64,StatsFuns.logaddexp,+,-Inf,0},SparseMatrixCSC{MarkovModels.Semiring{Float64,StatsFuns.logaddexp,+,-Inf,0},Int64}}:
 -0.693147       -Inf       -Inf
 -0.693147  -0.693147       -Inf
      -Inf  -0.693147  -0.693147

In [67]:
startstates = Dict(s => w for (s,w) in MarkovModels.nextemittingstates(initstate(fsm)))
endstates = Dict(s => w for (s,w) in MarkovModels.nextemittingstates(initstate(transpose(fsm))))

Dict{State,Float64} with 1 entry:
  State(2, 3, c, Link[...]) => -0.693147

In [68]:
ls_llh = [convert(Vector{LogSemiring{Float64}}, col) for col in eachcol(llh) ]

α = [sparse(zeros(LogSemiring{Float64}, S))]
for (s,w) in startstates 
    α[1][s.pdfindex] = LogSemiring{Float64}(w) * ls_llh[1][s.pdfindex]
end 

for n in 2:N 
    push!(α, (Aᵀ * α[end]) .* ls_llh[n] ) 
end

In [69]:
α

5-element Array{SparseVector{MarkovModels.Semiring{Float64,StatsFuns.logaddexp,+,-Inf,0},Int64},1}:
   [1]  =  0.39235
   [1]  =  -0.29094
  [2]  =  -0.383922
   [1]  =  0.0662038
  [2]  =  -0.310888
  [3]  =  -2.50271
   [1]  =  -1.05058
  [2]  =  -2.04096
  [3]  =  0.291114
   [1]  =  -1.85019
  [2]  =  0.406191
  [3]  =  1.38859

In [70]:
lnα

LoadError: UndefVarError: lnα not defined

In [40]:
ls_llh[1]

3-element Array{MarkovModels.Semiring{Float64,StatsFuns.logaddexp,+,-Inf,0},1}:
   2.2832521142797293
   -1.831520532078208
 -0.42362611074623524

In [41]:
A

3×3 SparseMatrixCSC{MarkovModels.Semiring{Float64,StatsFuns.logaddexp,+,-Inf,0},Int64} with 5 stored entries:
  [1, 1]  =  -0.693147
  [1, 2]  =  -0.693147
  [2, 2]  =  -0.693147
  [2, 3]  =  -0.693147
  [3, 3]  =  -0.693147

In [ ]:
statemap